# Job Assistant 

Recent advancements in LLM has proven how powerful these models are and how far these technology can go. In the previous post, I talked about the magic technique called "prompting technique" that can harness the power of LLMs without going through an expensive fine-tuining process. One of the latest technique ReAct, gives the model ability to reason and act extending their use cases. In this post, I am going to build a simple Job assistant using a LangChain agent. So, let's dive in!

First, we should load our resume so the model could analyze it and find the core competency of the candidate. In the following I imported PyPDFium2Loader which is faster than PyPDFLoader, then load the pdf file of the resume.

In [2]:
import langchain.llms.openai
from openai import OpenAI

from langchain.document_loaders import PyPDFium2Loader
from langchain.agents import AgentType, initialize_agent, load_tools
import os
import json

In [ ]:
loader = PyPDFium2Loader("docs/Resume.pdf")
pages = loader.load()

In [5]:
page_content = []
for page in pages:
    page_content.append(page.page_content)
all_pages = "".join(page_content) 

After we sucesfully loaded the pdf file of the resume. We ask our model to analyze our resume and find the best core competency based on the experiences and the skill sets mentioned in the resume. 

In [26]:
os.environ["OPENAI_API_KEY"] = "open_ai key"
os.environ["SERPAPI_API_KEY"] = "serp_api key"

In [8]:
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful job assistant."},
    {"role": "user", "content": f"Please define the best job role for the candidate in this resume: {all_pages}. Do not include any names in your response. \n\nJob role |  Core competencies | Seniority level"}   
  ]
)

In [9]:
result = response.choices[0].message.content

In [11]:
responses = rresult.lower().split("\n")

In [17]:
for res in responses:
    if res.startswith("job role"):
        job_role = res[10:]
    if res.startswith("seniority level"):
        seniority_level = res[17:]

So far so good! We have extracted the core competency of the candidate. Here, ReAct framework comes in. The underlying model needs to reason on the input data and act accordingly on the external sources and reason again and act acoordignly util it reeaches the answer or the stop condition. As a prompt, we add a context to our question (i.e., the core competency derived from the resume). We also need to create a ReAct agent to act on behalf of the model. For this, we define a LangChain agent that has access to a job posting website (Google Job in this example).

In [22]:
llm = langchain.llms.openai.OpenAI(temperature=0.1)

In [27]:
tools = load_tools(["google-jobs"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [33]:
from langchain.prompts import PromptTemplate

multi_var_prompt = PromptTemplate(
    input_variables=["seniority_level", "job_role","country"], 
    template="""

User prompt: What is the best job posting for a {seniority_level} {job_role} in {country}?

Output:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(seniority_level=seniority_level, job_role=job_role, country="Canada")

In [ ]:
agent.run(prompt)